# Генерация контекста для инструкций

На [предыдущем шаге](https://github.com/EliseevVadim/InstructionsGenerator/blob/main/generated_instructions_analysis.ipynb) мы сгенерировали инструкции по текстам, полученным после парсинга страниц сайта [Донецкого Государственного университета](https://donnu.ru/), а также провели повторную генерацию инструкций для страниц, по которым изначально было сгенерировано менее трех инструкций

Поскольку модель-генератор, которую мы планируем дообучить с помощью [LoRA](https://huggingface.co/docs/text-generation-inference/conceptual/lora) будет работать с векторной базой знаний об университете с помощью механизма [RAG](https://habr.com/ru/articles/779526/), то будет полезно дообучить модель не столько отвечать на вопросы пользователя, сколько выделять релевантную информацию из предложенных фрагментов документов, полученных благодаря векторному поиску по базе знаний 

Поэтому на данном шаге мы составим векторную базу знаний по данным, полученным после парсинга страниц сайта ДонГУ, после чего для каждого сгенерированного вопроса найдем `3` наиболее релевантных чанка с помощью векторного поиска, реализованного в [FAISS](https://faiss.ai/), и добавим их в наш датасет как поле `context`

Благодаря этому, во время дообучения, мы будет в качестве входа подавать кобминированный промпт состоящий из вопроса пользователя, контекстных чанков и базовых инструкций для модели, что позволит ей лучше адаптироваться к пайплайну работы с использованием [RAG](https://habr.com/ru/articles/779526/)

## Импорт необходимых библиотек и модулей

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd

In [4]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

In [5]:
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
from core.embedders.giga_embeddings import get_embeddings
from core.utils.common import init_random_seed
from core.utils.data_analysis import *

Для воспроизводимости экспериментов

In [7]:
init_random_seed(42)

## Загрузка и предобработка исходных данных

Проинициализируем названия необходимых нам LLM и пути к файлам с текстами и сгенерированными инструкциями

In [8]:
tokenizer_name_or_path = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
embedder_model_name_or_path = 'ai-sage/Giga-Embeddings-instruct'
relevant_texts_path = 'data/input/relevant_texts.csv'
instructions_path = 'data/output/generation_log.json'

Загрузим датафрейм с релевантными текстами, он нам понадобится для построения векторной базы знаний

In [9]:
relevant_texts = pd.read_csv(relevant_texts_path, index_col=0)
relevant_texts.sample(10)

,id,filename,content,size,rounded_size_kb,last_updated,already_evaluated,gemma2-9b-it,llama-3.1-70b-versatile,llama-3.3-70b-versatile,average_score
2068,2480,Физическая культура и спорт_ теория и практика,Русский,14,0,18-11-2021,True,2.0,2.0,3.0,2.333333
2379,864,Информация о направлениях и результатах научно...,Содержание\n\nИнформация о сроке действия госу...,138309,135,18-09-2024,False,5.0,5.0,5.0,5.000000
1719,2067,Редакционная политика,Редакционная политика\n\nАвторское право.\n\nС...,4836,5,27-02-2024,True,3.0,4.0,4.0,3.666667
2674,2556,Хвостова Елена Николаевна,Хвостова Елена Николаевна\n\nДолжность:\n\nста...,12131,12,07-06-2024,False,5.0,5.0,5.0,5.000000
2495,1462,Направления работы центра,ПСИХОЛОГИЧЕСКОЕ КОНСУЛЬТИРОВАНИЕ\n\nПсихологи ...,15842,15,28-04-2023,False,5.0,5.0,5.0,5.000000
1979,2385,Учебная деятельность,Торжественная церемония вручения дипломов\n\nК...,6638,6,05-12-2019,True,4.0,4.0,4.0,4.000000
564,635,Есенинский центр,В марте 2018 года ректором ДонНУ С. В. Беспало...,4913,5,29-08-2024,True,4.0,4.0,3.0,3.666667
1566,1898,Преподавательский состав,ПРОФЕССОРСКО-ПРЕПОДАВАТЕЛЬСКИЙ СОСТАВ КАФЕДРЫ\...,2005,2,14-11-2019,True,3.0,4.0,4.0,3.666667
1011,1198,Кудрейко Ирина Александровна,Кудрейко Ирина Александровна\n\nДолжность:\n\n...,7801,8,31-08-2022,True,4.0,4.0,4.0,4.000000
562,632,Еропутова Ольга Александровна,Еропутова Ольга Александровна\n\nДолжность:\n\...,9780,10,08-05-2024,True,4.0,5.0,5.0,4.666667


Проверим наличие дублирующихся записей во фрейме, поскольку это является принципиальным моментом для построения векторной базы знаний (при наличии дубликатов, векторный поиск будет выдавать одинаковые чанки для некоторых вопросов, что не является объективным результатом)

In [10]:
duplicates = relevant_texts[relevant_texts.duplicated(subset=['content', 'last_updated'])]

print(f"Было найдено число дублирующихся записей: {duplicates.shape[0]}")

Было найдено число дублирующихся записей: 358


Удалим их, и посмотрим на число оставшихся записей

In [11]:
relevant_texts = relevant_texts.drop_duplicates(subset=['content', 'last_updated'], keep='first')

print(f"После удаления дублирующихся записей осталось записей: {relevant_texts.shape[0]}")

После удаления дублирующихся записей осталось записей: 1979


## Загрузка необходимых моделей и токенизатора

Определим устройство, на котором будет происходить работа модели-векторизатора

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Загрузим токенизатор модели `Llama-3.1-8B-Instruct`. Он нам необходим для разбиения документов на чанки - мы будем использовать чанки размером по 500 токенов с перекрытием в 50 (10%). Мы используем именно этот токенизатор, поскольку планируется дообучить либо модель `Llama-3.1-8B-Instruct` либо другую из семейства Llama, а использование токенизаторов от других моделей сделает разбиение малопригодным в контексте управления размером данных для дообучения 

In [13]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Для построения эмбеддингов воспользуемся моделью [`Giga-Embeddings-instruct`](https://huggingface.co/ai-sage/Giga-Embeddings-instruct) от Сбера. Данная модель имеет всего 2.5 миллиарда параметров, что делает ее пригодной для использования в домашних условиях. Кроме того, модель изначально была обучена на данных на русском языке и несмотря на свой относительно небольшой размер демонстрирует высокое качество эмбеддингов - модель занимает вторую строчку в [либерборде](https://huggingface.co/spaces/mteb/leaderboard) ruMTEB на HuggingFace *(на момент 01.01.2025)*

In [14]:
embedder = AutoModel.from_pretrained(embedder_model_name_or_path, trust_remote_code=True, device_map=device)
embedder

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.96s/it]


GigarEmbedModel(
  (latent_attention_model): LatentAttentionModel(
    (cross_attend_blocks): ModuleList(
      (0-1): 2 x PreNorm()
    )
  )
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-26): 27 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (in

## Построение векторной базы знаний

Конвертируем содержимое релевантных текстовых файлов в формат, пригодный для построения векторной базы знаний

In [15]:
docs = [
    Document(
        id=row['id'],
        page_content=row['content'],
        metadata={
            "id": row['id'],
            "filename": row['filename'],
            "last_updated": row['last_updated']
        }
    )
    for _, row in relevant_texts.iterrows()
]


assert len(docs) == relevant_texts.shape[0]
print("OK")

OK


Разобьем документы на чанки по 500 токенов с перекрытием в 50

In [16]:
%%time
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""],
    add_start_index=True,
    length_function=lambda text: len(tokenizer.encode(text, add_special_tokens=False))
)

chunks = text_splitter.split_documents(docs)

CPU times: total: 51.2 s
Wall time: 51.8 s


Посмотрим на число получившихся чанков

In [17]:
print(f"Всего было получено чанков: {len(chunks)}")

Всего было получено чанков: 9935


Проинициализируем модель-эмбеддер в формате, пригодном для построения векторной базы знаний

In [18]:
embeddings = get_embeddings(embedder)

Построим саму базу знаний

In [19]:
%%time
db = FAISS.from_documents(chunks, embeddings)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


CPU times: total: 1h 3min 58s
Wall time: 1h 4min 56s


И сохраним ее

In [20]:
db.save_local('data/output/vector_db_for_context_500')

## Проверка качества полученных эмбеддингов

Проверим полученные эмбеддинги на ряде тестовых вопросов

In [21]:
question = "Кто такой Бондаренко Виталий?"
passages = db.similarity_search_with_relevance_scores(question, k=3)
passages

[(Document(id='833e2c25-a25e-4c80-814e-154465a5af4d', metadata={'id': 263, 'filename': 'Бондаренко Виталий Иванович ', 'last_updated': '14-09-2024', 'start_index': 0}, page_content='Бондаренко Виталий Иванович\n\nДолжность:\n\nдоцент кафедры компьютерных технологий\n\nТелефон:\n\n3523, 3570\n\nКабинет:\n\n404\n\nПеречень преподаваемых дисциплин:\n\nуправление проектированием информационных систем;\n\nинтеллектуальный анализ данных;\n\nбазы данных;\n\nСУБД Oracle;\n\nбезопасность и защита информации в информационных системах;\n\nпрограммирование;\n\nобъектно-ориентированное программирование;\n\nруководство выпускными квалификационными работами.\n\nУровень образования:\n\nвысшее, специалитет. Донецкий государственный университет, 1993.\n\nНаименование направления подготовки или специальности:\n\nспециальность: «Физика»\n\nКвалификация:\n\nфизик-инженер\n\nУченая степень:\n\nкандидат технических наук\n\nУченое звание:\n\nдоцент по научной специальности 2.3.1. Системный анализ, управление 

Как видно, с помощью векторного поиска было получено 3 чанка, в первом из которых содержится ответ на заданный вопрос. При инференсе модели мы будем использовать чанки большего размера, тем самым получая более развернутые ответы, а в данном случае размер чанков подходит под целевые ответы, полученные при генерации инструкций

Рассмотрим еще несколько вопросов

In [22]:
question = "Адрес юридического факультета"
passages = db.similarity_search_with_relevance_scores(question, k=3)
passages

[(Document(id='37c91c11-fb89-411c-aa9a-be5fc7879e6e', metadata={'id': 1149, 'filename': 'Контакты юридического факультета ', 'last_updated': '25-12-2019', 'start_index': 0}, page_content='Адрес для переписки и местоположение:\n\n283050, г. Донецк, пр. Ватутина,1а\n\nТелефон +380 62\xa0302-09-50 (приёмная декана)\n\nE-mail:\n\nfcl.jur@ donnu.ru\n\nФакультет в социальных сетях:\n\nhttps://vk.com/fcl_law'),
  0.25251620076522796),
 (Document(id='2a440ff7-0af2-4c61-8d85-e8657b29b54f', metadata={'id': 1456, 'filename': 'Направления подготовки на юридическом факультете ', 'last_updated': '11-12-2018', 'start_index': 0}, page_content='Юридический факультет\n\nосуществляет обучение специалистов по направлению подготовки «Юриспруденция» по двум образовательным программам — 40.03.01 «Бакалавр» и 40.04.01 «Магистр». Формы обучения — очная, заочная и заочная с сокращенным сроком обучения на базе СПО (среднего профессионального образования).\n\nНа Юридическом факультете ГОУ ВПО «Донецкий национальн

In [23]:
question = "Где находится деканат физико-технического факультета?"
passages = db.similarity_search_with_relevance_scores(question, k=3)
passages

[(Document(id='a994d023-c6c4-45cf-ac23-15af18bb6ac5', metadata={'id': 1118, 'filename': 'Контакты ', 'last_updated': '15-02-2017', 'start_index': 0}, page_content='Адрес\n\n: 283001, г. Донецк, пр. Театральный, д. 13\n\nДеканат физико-технического факультета\n\nТелефон\n\n(062) 302-07-58\n\nEmail\n\ndonnu.phys@mail.ru\n\nКафедра общей физики и дидактики физики\n\nТелефон\n\n(062) 302-92-62\n\nEmail\n\nkofdonnu@mail.ru'),
  0.40046238648483623),
 (Document(id='1e79e128-b45e-4d12-9571-85295e42da48', metadata={'id': 103, 'filename': '_ Page 7 ', 'last_updated': '21-05-2016', 'start_index': 0}, page_content='Адрес для переписки: г. Донецк, ул.Университетская, 24\n\nМестоположение: г. Донецк, пр.Театральный, 13 (корпус IV)\n\nТел. +380 62 302-07-58\n\nE-mail:\n\ndonnu.phys@mail.ru\n\nhttp://vk.com/fcl_phys\n\nУчебно-методический сайт факультета\n\nhttps://sites.google.com/site/donnuftf\n\nДеканат заочного отделения\n\nhttps://sites.google.com/site/dzofizteh'),
  0.30030120178881836),
 (Docu

Во всех случаях найденные чанки содержат ответ на поставленный вопрос, что позволяет нам судить о достаточном качестве векторизатора

## Добавление контекста к инструкциям

Загрузим инструкции, полученные после повторной генерации [ранее](https://github.com/EliseevVadim/InstructionsGenerator/blob/main/generated_instructions_analysis.ipynb)

In [24]:
instructions = read_generation_log_as_frame(data_path=instructions_path)
instructions.sample(10)

,file_id,filename,input,output
2496,342,Видео _ Кафедра КТ,Какой тип контента представлен на странице Вид...,Видеоматериалы
22126,1086,Комарницкая Елена Владимировна,Какой стаж работы имеет Комарницкая Елена Влад...,Комарницкая Елена Владимировна имеет общий ста...
5201,775,Информация для студентов,Кто такой Пелашенко Алла Владимировна?,Пелашенко Алла Владимировна - старший преподав...
13827,1927,Преподавательский состав кафедры ВМиМПМ,Кто является старшим лаборантом на кафедре ВМи...,Петлицкая Оксана Николаевна
12085,1704,"Отчеты, доклады",Какие доклады и отчеты представлены на сайте Д...,Доклад ректора на Конференции работников и обу...
4165,627,Енакиевский учебно-консультационный центр,Какова история создания и работы Енакиевского ...,Енакиевский учебно-консультационный центр гото...
4339,648,Жуков Сергей Михайлович,Какие дисциплины преподает Жуков Сергей Михайл...,Жуков Сергей Михайлович преподает воспитательн...
8005,1128,Контакты библиотеки,Где находится научная библиотека Донецкого гос...,Научная библиотека Донецкого государственного ...
4955,732,Информация для преподавателей,Когда был введен режим электронного обучения и...,Режим электронного обучения и дистанционных об...
17133,2372,Учебная деятельность,Кто такой Гулина Светлана Николаевна?,Гулина Светлана Николаевна - старший преподава...


Для добавления поля с контекстом скопируем фрейм и удалим записи с пропусками

In [25]:
instructions_with_context = instructions.copy()
instructions_with_context = instructions_with_context.dropna()

Добавим поле с контекстом, применив поиск по векторной базе знаний

In [26]:
%%time
instructions_with_context['context'] = instructions_with_context['input']\
                                        .apply(lambda question: retrieve_context_by_question(db, question, 3))

CPU times: total: 24min 9s
Wall time: 24min 34s


In [27]:
instructions_with_context.sample(10)

,file_id,filename,input,output,context
9151,1277,Ломов Дмитрий Александрович,Какое образование имеет Дмитрий Александрович ...,Дмитрий Александрович Ломов имеет высшее образ...,[Document(id='90d2a496-1890-4a22-8510-da3c46ec...
1917,249,Биологический факультет,С какими учреждениями сотрудничает Биологическ...,Биологический факультет сотрудничает и имеет н...,[Document(id='d5b3e29c-80c3-44b4-ae63-29b2d1c1...
12628,1787,Полякова Виктория Александровна,Кто такая Виктория Александровна Полякова?,Виктория Александровна Полякова - старший преп...,[Document(id='b7ee226f-7980-403c-b04a-8fabf2ab...
21163,769,Информация для студентов,Кто работает на кафедре мировой и отечественно...,На кафедре мировой и отечественной культуры ра...,[Document(id='0c16ce62-f3de-4f68-bc28-89ed746f...
14195,2008,Радиофизика и электроника РФ,Кем являются востребованные специалисты в совр...,"«Технари», обладающие фундаментальными знаниями",[Document(id='057ab330-7a2d-4a33-a16a-af1ba02f...
18696,2582,Цупило Ираида Александровна,Какой уровень образования имеет Цупило Ираида ...,Цупило Ираида Александровна имеет высшее образ...,[Document(id='de80c47b-ccb5-4077-ba92-94f3e35c...
6575,939,Кафедра административного и финансового права,Какой уровень подготовки имеют выпускники кафе...,Выпускники кафедры административного и финансо...,[Document(id='6711b2af-8a75-40f8-ab0a-9cfeb798...
1084,122,Абитуриенту,Как записаться на курс «Школа юного физика»?,"Чтобы записаться на курс «Школа юного физика»,...",[Document(id='4479feb2-2ea2-4181-bf1f-7111777a...
23675,1742,Петенко Ирина Валентиновна,Какая ученая степень у Петенко Ирина Валентино...,Петенко Ирина Валентиновна имеет ученую степен...,[Document(id='a47c552d-3ccb-43e4-b479-f34576a8...
21950,1043,Квач Юлия Александровна,Какие направления подготовки или специальности...,Квач Юлия Александровна имеет специальности: «...,[Document(id='58dffa71-1f97-43f5-9df2-ed5a9d4c...


Удалим столбцы `file_id` и `filename` как более нерелевантные

In [28]:
instructions_with_context = instructions_with_context.drop(['file_id', 'filename'], axis=1)
instructions_with_context.sample(10)

,input,output,context
23773,Кто является заведующей кафедрой и доцентом ка...,Подгайская Ирина Михайловна,[Document(id='db73d328-603a-40d1-83c8-4607b769...
17301,Какой график учебного процесса очной формы обу...,ГРАФИК учебного процесса очной формы обучения ...,[Document(id='e90f730c-65fc-4cd1-bb1f-30018dca...
3714,Какие медицинские справки необходимы для посту...,Медицинская справка для поступающих на направл...,[Document(id='6c75166c-287d-460b-8dcf-1396d0e5...
24769,Какой стаж работы имеет Слота Наталья Владимир...,Слота Наталья Владимировна имеет общий стаж ра...,[Document(id='b313fb11-1902-44b1-9296-0b5b8e7e...
4251,Какая квалификация у Ждановой Наталии Александ...,"Филолог, преподаватель английского языка и лит...",[Document(id='2dc447df-eac1-4ae5-ae91-e6c66b70...
13344,Кто такой Посредников Дмитрий Владимирович?,Посредников Дмитрий Владимирович - кандидат ис...,[Document(id='22a3f9e0-7499-41d3-97d4-8aa437dc...
11471,Кто является инициатором открытия Есенинского ...,"Член Общественной палаты Российской Федерации,...",[Document(id='18153920-523b-4a18-be0c-3c90398d...
13781,Кто такой Пивоварова Анастасия Сергеевна?,Пивоварова Анастасия Сергеевна - ассистент в Д...,[Document(id='6334a4f5-9ba4-4385-bfb3-68314333...
4737,В чем заключается значение издательства в унив...,Издательство является неотъемлемой частью унив...,[Document(id='51485c71-54ba-46d7-a10e-b7e117f7...
379,Какой стаж работы у Елены Владимировны Андриенко?,У Елены Владимировны Андриенко общий стаж рабо...,[Document(id='cfee7cf2-5d20-435a-877a-9d207fc0...


Сохраним полученный датасет в `csv` файл

In [29]:
instructions_with_context.to_csv('data/output/instructions_with_context.csv')

А также в формате `json`

In [30]:
with open('data/output/instructions_with_context.jsonl', 'w', encoding='utf-8') as file:
    instructions_with_context.to_json(file, indent=4, orient='records', force_ascii=False)